# ETL Pipeline
## Import data and store them in separated dataframes

In [1]:
import pandas as pd
import numpy as np
import math
import json
import datetime
get_ipython().run_line_magic('matplotlib', 'inline')

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Clean input data

Suspicious data has been seen in the profile dataframe. Customers with no gender and age of 118 years are being eliminated from the dataframe because of its suspiciously high age. 
### Profile Cleaning (part1)

In [2]:
# eliminate duplicate and NaN values of the profile dataframe
profile_clean = profile.dropna(subset=['income'],axis=0)
profile_clean = profile_clean.drop_duplicates()
profile_clean.head()

,gender,age,id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0


In [3]:
# Parse dates in column 'became_member_on'
profile_clean['days_being_members'] = datetime.datetime.today().date() - pd.to_datetime(profile_clean['became_member_on'], format='%Y%m%d').dt.date
profile_clean['days_being_members'] = profile_clean['days_being_members'].dt.days
profile_clean.head()

,gender,age,id,became_member_on,income,days_being_members
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1637
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1704
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,1352
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0,1428
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0,1518


In [4]:
profile_clean[['became_member_on','days_being_members']].head()

,became_member_on,days_being_members
1,20170715,1637
3,20170509,1704
5,20180426,1352
8,20180209,1428
12,20171111,1518


### Transcript Cleaning

In [5]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [6]:
# find droped ids in the profile dataframe to drop them in trqanscript dataframe
eliminated_ids = set(profile['id'].to_list()).difference(profile_clean['id'].to_list())
transcript_clean=transcript.drop(transcript.index[transcript['person'].isin(eliminated_ids)])
# rename person column to id
transcript_clean['id'] = transcript_clean['person']
transcript_clean.drop(['person'],axis=1,inplace=True)
# Create dummy variables for each event in the transcript dataframe
transcript_clean = pd.concat([transcript_clean, pd.get_dummies(transcript_clean['event'],drop_first=False)],axis=1)
# Reset index
transcript_clean.reset_index(drop=True,inplace=True)

In [7]:
### WARNING: this will take a while run only if necessary, clean jason file is already created in data folder
# divide value data into columns
transcript_clean['offer_id'] = np.nan
transcript_clean['amount'] = np.nan
for i in range(transcript_clean.shape[0]):
    col = list(transcript_clean.loc[i,'value'].keys())[0]
    col = col.replace(' ','_')
    val = list(transcript_clean.loc[i,'value'].values())[0]
    transcript_clean.loc[i,col] = val
del transcript_clean['value']
# Fill amount NaN with 0 values
transcript_clean['amount'].fillna(0.0,inplace=True)
transcript_clean.head()

,event,time,id,offer completed,offer received,offer viewed,transaction,offer_id,amount
0,offer received,0,78afa995795e4d85b5d9ceeca43f5fef,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0
1,offer received,0,e2127556f4f64592b11af22de27a7932,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.0
2,offer received,0,389bc3fa690240e798340f5a15918d5c,0,1,0,0,f19421c1d4aa40978ebb69ca19b0e20d,0.0
3,offer received,0,2eeac8d8feae4a8cad5a6af0499a211d,0,1,0,0,3f207df678b143eea3cee63160fa8bed,0.0
4,offer received,0,aa4862eba776480b8bb9c68455b8c2e1,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0


In [8]:
# Merge clean transcript data with clean profile data
transcript_merge = transcript_clean.merge(profile_clean,on='id')

In [9]:
# map integer number to each customer id
def id_mapper():
    coded_dict = dict()
    cter = 1
    id_encoded = []
    
    for val in transcript_merge['id']:
        if val not in coded_dict:
            coded_dict[val] = cter
            cter+=1
        
        id_encoded.append(coded_dict[val])
    return id_encoded
id_encoded = id_mapper()
del transcript_merge['id']
transcript_merge['customer_id'] = id_encoded
transcript_merge.head()

,event,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,gender,age,became_member_on,income,days_being_members,customer_id
0,offer received,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,F,75,20170509,100000.0,1704,1
1,offer viewed,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,F,75,20170509,100000.0,1704,1
2,transaction,132,0,0,0,1,NaN,19.89,F,75,20170509,100000.0,1704,1
3,offer completed,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,F,75,20170509,100000.0,1704,1
4,transaction,144,0,0,0,1,NaN,17.78,F,75,20170509,100000.0,1704,1


In [10]:
# link index with ids
offer_index = {}
for i in portfolio.index:
    key = portfolio.loc[i,'id']
    val = i
    offer_index[key]=val


In [11]:
### WARNING: this will take a while run only if necessary, clean jason file is already created in data folder
# replace offer ide by index. 
for i in transcript_merge.index:
    if str(transcript_merge['offer_id'].iloc[i]) !=  "nan":
        transcript_merge.loc[i,'offer_id'] = offer_index[transcript_merge.loc[i,'offer_id']]
transcript_merge.head()

,event,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,gender,age,became_member_on,income,days_being_members,customer_id
0,offer received,0,0,1,0,0,3,0.00,F,75,20170509,100000.0,1704,1
1,offer viewed,6,0,0,1,0,3,0.00,F,75,20170509,100000.0,1704,1
2,transaction,132,0,0,0,1,NaN,19.89,F,75,20170509,100000.0,1704,1
3,offer completed,132,1,0,0,0,3,0.00,F,75,20170509,100000.0,1704,1
4,transaction,144,0,0,0,1,NaN,17.78,F,75,20170509,100000.0,1704,1


In [12]:
# Export cleaned json file
transcript_merge.to_json('data/transcript_profile_merge.json', orient='records', lines=True)

### Profile cleaning (part2)

In [13]:
# Generate clean profile and export it to json file in the data folder
new_profile = transcript_merge[['customer_id','income','gender','age','became_member_on','days_being_members']]
new_profile = new_profile.drop_duplicates()
new_profile.set_index(new_profile['customer_id'],drop=True,inplace=True)
del new_profile['customer_id']
new_profile.to_json('data/new_profile.json', orient='records', lines=True)

In [14]:
new_profile.head()

,income,gender,age,became_member_on,days_being_members
customer_id,,,,,
1,100000.0,F,75,20170509,1704
2,70000.0,M,68,20180426,1352
3,53000.0,M,65,20180209,1428
4,51000.0,M,58,20171111,1518
5,57000.0,F,61,20170911,1579


### Portfolio cleaning

In [19]:
portfolio_clean = portfolio.copy()
#generate dummy variables for each type of channels
portfolio_clean['web'] = portfolio_clean['channels'].apply(lambda x: 1 if 'web' in x else 0)
portfolio_clean['email'] = portfolio_clean['channels'].apply(lambda x: 1 if 'email' in x else 0)
portfolio_clean['mobile'] = portfolio_clean['channels'].apply(lambda x: 1 if 'mobile' in x else 0)
portfolio_clean['social'] = portfolio_clean['channels'].apply(lambda x: 1 if 'social' in x else 0)

# generate dummy variables for each offer type
offer_type = pd.get_dummies(portfolio_clean['offer_type'])

#Drop and concatenate portfolio with offer types
portfolio_clean.drop(['channels'], axis=1, inplace=True)
portfolio_clean = pd.concat([portfolio_clean, offer_type], axis=1, sort=False)
portfolio_clean.to_json('data/portfolio_clean.json', orient='records', lines=True)
portfolio_clean

,reward,difficulty,duration,offer_type,id,web,email,mobile,social,bogo,discount,informational
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1,1,0,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0,0,0,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0,1,0,0
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0,0,1,0
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,0,1,0
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,0,1,0
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,0,1,1,1,0,0,1
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,1,0,0
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,1,0,0,1,0
